# Numerical Anlysis for Non-Linear Optimization 


## First-Order Linear Equations

In the next week, we will study equation of the form: 

$$ 
x'(t) = f(t, x(t))
$$

where $f: (a,b) \times (c,d) \to \mathbb{B}$ is continuous, $-\infty \leq a<b \leq \infty$ and $-\infty \leq c,d \leq \infty$.


### Variation of Constants Formula 

Consider

$$
\begin{align}
x' &= p(t)x+q(t) \\
x(t_0) &= x_0
\end{align}
$$

where $p,q:(a,b) \to \mathbb{R}$ are continuous functions, $-\infty \leq a < b
\leq \infty$, $t_0 \in (a,b)$ and $x_0\in \mathbb{R}$. 

Then for $t\in(a,b)$, the solution of the above equation is given by

$$
x(t) = e^{\int_{t_0}^{t} p(\tau)\, d\tau} x_0 + e^{\int_{t_0}^{t} p(\tau)\, d\tau} \int_{t_0}^{t} e^{-\int_{t_0}^{s} p(\tau)\, d\tau} q(s)\, ds,

$$

#### Example

COnsider $p(\tau) = \tau$ and $q(\tau) = \sin(\tau)$. First, let's write a function to compute the solution of the above equation.


In [1]:
def phi(p, q, x0, t0, t):
    tau, s = var('tau s')
    # Compute the exponential integrating factor
    exp_int_p = exp(integrate(p(tau), (tau, t0, t)))
    
    # First term
    term1 = exp_int_p * x0
    
    # Inner integral inside the second term
    inner_exp = exp(-integrate(p(tau), (tau, t0, s)))
    inner_integrand = inner_exp * q(s)
    inner_integral = integrate(inner_integrand, (s, t0, t))
    
    # Second term
    term2 = exp_int_p * inner_integral
    
    return term1 + term2


Then, let's define the adeuacte parameters:

In [16]:
t, t0 = var('t t0')

assume( t0 > 0)
assume(t > t0)

p(t) = -2
q(t) = 140 + 20 * exp(-2*t)
x0 = 40
x = phi(p, q, x0, 0, t)
print(x.expand())

20*t*e^(-2*t) - 30*e^(-2*t) + 70


Finally, let's verify that the given function is a solution of the initial value problem.

In [11]:
diff(x,t).expand()

-40*t*e^(-2*t) + 80*e^(-2*t)

In [12]:
(p(t)*x + q(t)).expand()

-40*t*e^(-2*t) + 80*e^(-2*t)

In [13]:
x(t=0)

40

### Autonomous Equations

The equation $x'=f(x)$ is called autonomous because $f$ doesn't depends explicitly on $t$. We assume that $f:\mathbb{R} \to \mathbb{R}$ is continuously differentiable function. 